In [ ]:
import requests as req
import pandas as pd
import json
from datetime import datetime

# Utils 

## Consts

In [ ]:
DOMAIN = 'https://api.github.com/graphql'
TOKEN = ''
HEADERS = {
  'Authorization': f'bearer {TOKEN}',
  'Content-Type': 'application/json'
}
TODAY = datetime.now()
NUMBER_OF_REPOSITORIES=1000
PRIMARY_LANGUANGE = 'java'

## Functions

In [ ]:
def doPost(data : json)-> json:
  response = req.post(DOMAIN, headers=HEADERS, json=data)
  if response.status_code == 200:
    return response.json()

  raise Exception(f'Erro ao fazer requisição: {response.status_code} \n {response.text}')

def analisar_createdAt(repositories: list):
  data_frame = pd.DataFrame(repositories)
  created_dates = [datetime.strptime(date, '%Y-%m-%dT%H:%M:%SZ') for date in data_frame['createdAt']]
  age_years = [round((TODAY - date).days / 365, 2) for date in created_dates]
  return age_years

# REQUESTS

## QUERY

In [ ]:
QUERY_TEMPLATE = '''
query search($queryString: String!, $perPage: Int!, $cursor: String) {
  search(query: $queryString, type: REPOSITORY, first: $perPage, after: $cursor) {
    edges {
      node {
        ... on Repository {
          name
          nameWithOwner
          owner {
            login
          }
          stargazers {
            totalCount
          }
          createdAt
          defaultBranchRef {
            name
            target {
              ... on Commit {
                history(first: 5) {
                  edges {
                    node {
                      message
                      committedDate
                      author {
                        name
                        email
                        date
                      }
                    }
                  }
                }
              }
            }
          }
          releases {
            totalCount
          }
        }
      }
    }
    pageInfo {
      endCursor
      hasNextPage
    }
  }
}
'''

In [ ]:
per_page = 50
cursor = None
query_string = "language:java"
repositories = []

while len(repositories) < NUMBER_OF_REPOSITORIES:
    variables = {
        "queryString": query_string,
        "perPage": per_page,
        "cursor": cursor
    }
    
    data = doPost(data={'query': QUERY_TEMPLATE, 'variables': variables})

    if 'errors' in data:
        print("GraphQL query failed:", data['errors'])
        break
    
    # print(data)
    for edge in data['data']['search']['edges']:
        repositories.append(edge['node'])

    if data['data']['search']['pageInfo']['hasNextPage']:
        cursor = data['data']['search']['pageInfo']['endCursor']
    else:
        break

print("Total repositories: ", len(repositories))
print("Cursor: ", cursor)
print("Per page: ", per_page)

data_brutus = pd.DataFrame(repositories)
data_brutus.to_csv('../dataset/dados_base.csv', index=False, sep=';')


## data processing

In [ ]:
dataFrame_tratado = pd.DataFrame()
dataFrame_tratado['Repositorio'] = [repo.get('nameWithOwner') for repo in repositories]
dataFrame_tratado['Nome'] = [repo.get('name') for repo in repositories]
dataFrame_tratado['Estrelas'] = [repo.get('stargazers', {}).get('totalCount', 0) if isinstance(repo, dict) else 0 for repo in repositories]
dataFrame_tratado['Anos'] = analisar_createdAt(repositories)
dataFrame_tratado['nº Releases'] = [repo.get('releases').get('totalCount',0) if isinstance(repo, dict) else 0 for repo in repositories]

dataFrame_tratado.head()
dataFrame_tratado.to_csv('../dataset/dados_tratados.csv', index=False, sep=';')